# BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding

- **BERT**: **B**idirectional **E**ncoder **R**epresentations from **T**ransformers의 약자로, Transformer의 양방향 Encoder를 활용한 언어 모델

# 0. Abstract

- BERT에는 pretraining과 fine-tuning 두 가지 과정으로 학습한다.
    1. **pretraining**
        - unlabeled text 데이터를 이용한다.
        - bidirectional representatiions를 얻는다. (text의 양방향 정보를 모두 활용한다.)
    2. **fine-tuning**
        - downstream task(구체적으로 풀고 싶은 문제)에 알맞은 labeled 데이터를 이용한다.
        - pretrained model에서 오로지 하나의 layer만을 추가한다.

- QA(Question Answering)와 NLI(Natural Language Inference)에서 SOTA를 달성했다.
    - QA: input으로 질문과 지문이 주어지면 output으로 질문에 대한 정답을 도출하는 task
    - NLI: input으로 전제와 가설이 주어지면 output으로 가설이 참인지 거짓인지 도출하는 task

- 그 외에도 여러 task에서 높은 성능을 보였다.

# 1. Introduction

- BERT는 **Bidirectional Transformer**를 이용한다.
    - 저자들은 Unidirectional LM(Language Model)은 representation을 형성하는 데에 제한이 있다고 주장한다.
        - 예: GPT는 Transformer의 Decoder 부분을 활용하는데, 이는 left-to-right 구조를 가진다.
        - 특히 QA 같은 task에서는 양쪽 방향의 context를 통합하는 것이 중요하다.
    - 이에 따라 저자들은 BERT에 **MLM(Masked Language Model)**을 제안한다.
        - MLM은 input을 무작위로 mask하고, context(output)가 masked된 원래 단어를 예측한다.
    - **MLM**에는 **Bidirectional Transformer**를 사용한다.
        - 이는 양방향 문맥을 종합하여 representations를 만들 수 있도록 한다.

- 이 논문의 기여(Contributions)는 다음과 같다.
    1. Language representations에는 **bidirectional pre-training**이 중요함을 보여준다.
    2. pre-trained representations는 많은 복잡한 기술의 필요성을 줄여준다는 것을 보여준다.

# 2. Related Work

<img src="01.png" width=70% height=70%>

- **GPT**: OpenAI에서 개발한 언어 모델로, left-to-right Transformer 구조를 가지고 있다.
- **ELMo**: LSTM을 이용한 각각의 Forward(left-to-right) 모델과 Backward(right-to-left) 모델을 모두 활용하여 만든 언어 모델이다.
- **BERT**는 위의 두 모델과는 달리 하나의 bidirectional Transformer 구조를 가지는 모델이다.

# 3. BERT 

- BERT는 1) **pre-training**과 2) **fine-tuning** 두 가지 과정으로 나뉜다.
    - 이때 **downstream task**가 무엇이든 pre-trained parameters는 같으나, fine-tuning만 달라진다.

<img src="02.png" width=70% height=70%>

- **Model Architecture**
    - BERT는 **Multi-layer bidirectional Transformer Encoder**를 이용한다.
    - 다음과 같이 정의하자.
        - $L$: the number of layers
        - $H$: hidden size
        - $A$: the number of self-attention heads
    - **BERT BASE**
        - OpenAI에서 만든 GPT와 같은 model size가 되도록 설계
        - $L=12, H=768, A=12$
        - Total parameters: 110M
    - **BERT LARGE**
        - $L=24, H=1024, A=16$
        - Total parameters: 340M

<img src="03.png" width=20% height=20%>

- **Input/Output Representations**
    - BERT는 다양한 downstream tasks(풀고자 하는 문제)를 수행하기 위해 하나의 **input sequence**에 **single sentence**와 **pair of sentences**가 모두 들어갈 수 있도록 하였다.
        - **sentence**: 연속된 text (실제 우리가 아는 문장과는 다름. 문장의 일부일 수도 있고, 여러 문장일 수도 있음.)
        - **sequence**: BERT의 input token sequence (single sentence 또는 pair of sentences)
    - Word Embedding으로는 30,000 token 단어들이 존재하는 WordPiece embeddings를 사용함.
    - BERT에 사용되는 다양한 Token들은 다음과 같다.
        - **[CLS]**: input sequence의 시작 Token
        - **[SEP]**: pair of sentences가 input으로 들어갈 경우, 두 문장 사이에 들어가는 Token
        - **[C]**: [CLS] Token에 대응되는 final hidden vector(output)
            - [C]는 추후에 classification을 위해 sequence representation을 모아주는 역할을 한다.
        - **E**: Input embedding
        - **T**: final hidden vector

<img src="04.png">

- Input representations는 **Token embedding**, **segment embedding**, **position embedding**의 **합**으로 구성된다.
    - **Token embedding**: 각 token의 기본 embedding(WordPiece embedding)
    - **Segment embedding**: pair of sentences(A sentence, B sentence)를 구분해주는 embedding
    - **Position embedding**: Transformer와 동일한 positional embedding
    - WordPiece에서는 -ing와 같은 접미사를 다른 단어로 취급하고 embedding한다. (##ing)

<img src="05.png">

## 3.1. Pre-training BERT

### Task #1: Masked LM (MLM)

- 직관적으로 left-to-right 또는 right-to-left보다는 **bidirectional**하게 문맥을 파악하는 것이 언어 모델에서 더 좋은 성능을 낼 것으로 보인다.
- 그렇다면 bidirectional model을 학습시키기 위해서는 어떻게 해야 할까? $\rightarrow$ **MLM(Masked Language Model)** 이용!
    - **MLM(Masked LM)**: 다음과 같은 과정을 따른다.
        1. input token 중 15%를 mask한다.
        2. mask된 token과 대응되는 final hidden vector에 softmax를 적용하여 원래 단어를 예측하게 한다. 손실함수는 **cross entropy loss**를 이용한다.
    - pre-training에서의 mask의 단점: fine-tuning에는 **[MASK] token**이 없어서 pre-training과 fine-tuning 사이에 mismatch가 발생한다.
    - 이를 해결하기 위해 mask할 token는 다음을 따른다.
        - **80%**: **[MASK] token**으로 mask
            - 예) my dog is [MASK]
        - **10%**: **random token**으로 mask를 대체함.
            - 예) my dog is cat
        - **10%**: mask하기 전 원래 token을 그대로 사용함.(**unchanged token**) (단, 예측은 그대로 진행)
            - 예) my dog is cute

### Task #2: Next Sentence Prediction (NSP)

- QA(Qusetion Answering)과 NLI(Natural Language Inference)는 두 문장 사이의 관계를 이해하는 것이 중요하다.
- 보통 NLM(Natural Language Model)은 두 문장 사이의 관계를 파악하지 못한다.
- BERT는 어떤 과정을 통해 관계를 파악하려고 했을까? $\rightarrow$ **NSP(Next Sentence Prediction)** task 수행!
    - **NSP(Next Sentence Prediction)**: 두 문장이 바로 이어지는 문장인지 예측하는 것
        - 만약 두 문장 A, B를 input으로 넣으려고 한다면, B문장은 다음과 같이 설정한다.
            - **50%**: 실제로 이어지는 문장 (label: **IsNext**)
            - **50%**: 실제로 이어지지 않는 random한 문장 (label: **NotNext**)
        - 이때 **[C]** token이 label을 예측하도록 유도한다.
        - 아주 간단한 아이디어이지만 QA, NLI에 상당히 큰 도움이 되었다!

### Pre-training data

- Pre-training data로 1) BooksCorpus(800M words), 2) English Wikipedia(2,500M words)를 사용하였다.

## 3.2. Fine-tuning BERT

- 저자들은 **self-attention** mechanism이 많은 downstream tasks를 수행할 수 있도록 도와주기 때문에 fine-tuning이 쉽다고 주장한다.
    - 기존 모델은 다음과 같은 두 과정을 거쳐야 했다.
        1. text를 독립적으로 인코딩
        2. bidirectional cross attention 적용
    - 그러나 self-attention은 이 과정을 한번에 처리하도록 한다.

- fine-tuning을 통해 다음과 같은 downstream task를 처리할 수 있다.
    1. Sentence Pair Classification Tasks: 두 문장을 보고 분류 작업을 수행
        - 예: QNLI: Sentence 1은 Question, Sentence 2는 Answer로 구성. Answer가 적절한지 아닌지 [C] token이 분류한다.
    2. Single Sentence Classification Tasks: 한 문장의 분류 작업을 수행
        - 예: SST-2: 문장으로부터 감정을 [C] token이 분류한다.
    3. Question Answering Tasks: Question과 Paragraph가 주어지면 Paragraph를 보고 Question에 대한 답을 도출해내는 작업을 수행
        - 예: SQuAD v1.1: 질문과 지문, 정답이 존재하는 문제
    4. Single Sentence Tagging Tasks: 한 문장으로부터 각 token에 대응되는 tag를 찾는 작업을 수행
        - 예: CoNLL-2023 NER: 문장에서 이름을 찾는 문제, 형태소 분석 문제 등

<img src="06.png" width=70% height=70%>

# 4. Experiments

- **GLUE(General Language Understanding Evaluation) benchmark**: 다양한 자연어 처리 작업을 포함하는 평가 도구
    - 다음 표는 BERT의 GLUE 평가 지표를 보여준다.

<img src="07.png">

- 그 당시 BERT는 NLP 분야에서 SOTA를 달성하였다.

**[참고 자료]**
1. BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding 논문: https://arxiv.org/abs/1810.04805
2. 08-5: BERT, 고려대학교 산업경영공학부 DSBA 연구실, Youtube: https://www.youtube.com/watch?v=IwtexRHoWG0&t=1870s
3. 17-02 버트(Bidirectional Encoder Representations from Transformers, BERT), wikidocs: https://wikidocs.net/115055
4. Attention Is All You Need 논문: https://arxiv.org/abs/1706.03762